In [ ]:
import pandas as pd
import chardet
import os
from tqdm import tqdm
import time

In [ ]:
file_path = '/home/marceloborges/Datasets/CAGED/tratada/bs_tratado.csv'

In [ ]:
dimension_path = '/home/marceloborges/Datasets/CAGED/NOVO CAGED/Layout Não-identificado Novo Caged Movimentação.xlsx'

In [ ]:
num_bytes_to_read = 10000

In [ ]:
with open(file_path, 'rb') as f:
    result = chardet.detect(f.read(num_bytes_to_read))

print(result['encoding'])

In [ ]:
lines_to_display = 10  # Número de linhas para exibir antes e depois da linha problemática

with open(file_path, 'r', encoding=result['encoding']) as file:
    all_lines = file.readlines()
    start = max(3633790 - lines_to_display, 0)
    end = min(3633790 + lines_to_display, len(all_lines))
    
    for i in range(start, end):
        print(f"Line {i}: {all_lines[i].strip()}")

In [ ]:
header_str = "competênciamov;região;uf;município;seção;subclasse;saldomovimentação;cbo2002ocupação;categoria;graudeinstrução;idade;horascontratuais;raçacor;sexo;tipoempregador;tipoestabelecimento;tipomovimentação;tipodedeficiência;indtrabintermitente;indtrabparcial;salário;tamestabjan;indicadoraprendiz;origemdainformação;competênciadec;competênciaexc;indicadordeexclusão;indicadordeforadoprazo;unidadesaláriocódigo;valorsaláriofixo"

with open(file_path, 'r', encoding=result['encoding']) as file:
    lines = file.readlines()

# Mantenha apenas as linhas que não são iguais ao cabeçalho
lines_to_keep = [line for line in lines if header_str not in line]

# Escreva o arquivo novamente com as linhas filtradas
with open(file_path, 'w', encoding=result['encoding']) as file:
    file.writelines(lines_to_keep)


In [ ]:
with open(file_path, 'r', encoding=result['encoding']) as file:
    lines = file.readlines()

lines_to_keep = [lines[0]]  # Mantendo a primeira linha como cabeçalho original
excluded_lines = []

for line in lines[1:]:
    first_column = line.split(';')[0].strip()
    if first_column.isdigit():
        lines_to_keep.append(line)
    else:
        excluded_lines.append(line)

# Reescrevendo o arquivo com as linhas filtradas
with open(file_path, 'w', encoding=result['encoding']) as file:
    file.writelines(lines_to_keep)


In [ ]:
problematic_lines = []
valid_lines = []

with open(file_path, 'r', encoding=result['encoding']) as file:
    lines = file.readlines()

    for line in lines:
        # Verifica se a primeira coluna (antes do primeiro ';') é numérica
        first_column = line.split(';')[0]
        if not first_column.isdigit():
            problematic_lines.append(line)
        else:
            valid_lines.append(line)

# Salva as linhas problemáticas em um arquivo separado
with open("problematic_lines.txt", 'w', encoding=result['encoding']) as file:
    file.writelines(problematic_lines)

# Escreve de volta as linhas válidas no arquivo original
with open(file_path, 'w', encoding=result['encoding']) as file:
    file.writelines(valid_lines)



In [ ]:
df = pd.read_csv(file_path, sep=';',encoding=result['encoding'])

In [ ]:
df.columns

In [ ]:
df['tipomovimentação'].unique()

In [ ]:
df.to_csv('exemplo.csv', index=False)

In [ ]:
abas = pd.ExcelFile(dimension_path)

In [ ]:
dimension={}
for sheet_name in abas.sheet_names:
    if not sheet_name=='Layout':
        dimension[sheet_name] = abas.parse(sheet_name)

In [ ]:
if not os.path.exists('dataframes_csv'):
    os.mkdir('dataframes_csv')

In [ ]:
progress_bar = tqdm(total=len(dimension), desc="Processando")
for sheet_name, df in dimension.items():
    csv_filename = f'dataframes_csv/{sheet_name}.csv'
    df.to_csv(csv_filename, index=False)
    progress_bar.update(1)
    time.sleep(0.1)
progress_bar.close()

In [ ]:
df.columns()